In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../../../../src'))

from read_and_write_docs import read_jsonl, read_rds
from tokenize_and_score import load_model
from utils import apply_temp_doc_id, build_metadata_df
from n_gram_functions import (
    common_ngrams,
    pretty_print_common_ngrams,
    keep_before_phrase,
    score_phrases,
    add_pmf_column,
    score_phrases_no_context
)

In [3]:
tokenizer, model = load_model("/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct")

In [4]:
corpus = "Wiki"
data_type = "training"

known_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
known = read_jsonl(known_loc)
known = apply_temp_doc_id(known)

unknown_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
unknown = read_jsonl(unknown_loc)
unknown_df = apply_temp_doc_id(unknown)

metadata_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/metadata.rds"
metadata = read_rds(metadata_loc)
filtered_metadata = metadata[metadata['corpus'] == corpus]
agg_metadata = build_metadata_df(filtered_metadata, known, unknown)

In [5]:
problem_dataset_agg = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_agg.jsonl")
problem_dataset_profile = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_profile.jsonl")

In [6]:
diff_author_problems = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
diff_author_problems.sort_values(["highest_common_count"], ascending=[False], inplace=True)
diff_author_problems[diff_author_problems['highest_common_count'] >= 3].head(50)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
287,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_3,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
161,Caboga vs Chanakyathegreat,Caboga,Chanakyathegreat,caboga_text_5,chanakyathegreat_text_1,5,Ġ 2 0 0 8
262,Dweller vs Ecelan,Dweller,Ecelan,dweller_text_3,ecelan_text_2,5,.Ċ i Ġdon 't Ġthink
364,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_12,garda40_text_1,5,Ġ 2 0 1 1
365,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_2,garda40_text_1,5,Ġ 2 0 1 2
395,Gwen_Gale vs Habap,Gwen_Gale,Habap,gwen_gale_text_3,habap_text_1,5,", Ġbut Ġi Ġdon 't"
286,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_2,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
441,HeadleyDown vs Hipocrite,HeadleyDown,Hipocrite,headleydown_text_2,hipocrite_text_4,5,Ġon Ġthis Ġtalk Ġpage .Ċ
135,Britmax vs BrotherDarksoul,Britmax,BrotherDarksoul,britmax_text_1,brotherdarksoul_text_4,4,Ġis Ġno Ġneed Ġto
165,Chanakyathegreat vs Cla68,Chanakyathegreat,Cla68,chanakyathegreat_text_10,cla68_text_1,4,", Ġi Ġdon 't"


In [7]:
known_doc = "floquenbeam_text_4"
known_text = known[known['doc_id'] == known_doc].reset_index().loc[0, 'text']

unknown_doc = "fragments_of_jade_text_10"
unknown_text = unknown[unknown['doc_id'] == unknown_doc].reset_index().loc[0, 'text']

common = common_ngrams(known_text, unknown_text, 2, model, tokenizer)
pretty_print_common_ngrams(common, tokenizer=tokenizer, order='len_asc')

2-grams (37): [' and how', ' because it', ' do so', " don't", ' edits,', ' for the', ' here.\n', ' i was', " it's", ' me with', ' me,', ' me.\n', ' more than', ' not the', ' on the', ' one of', ' pages.\n', ' problems with', ' someone who', ' they are', ' to be', ' to discuss', ' trying to', ' unblocked', ' with me', ' you are', ', as', ', but', ', he', ', i', ', it', ', no', ', the', ', you', '.\ni', '.\nthe', 'when you']
3-grams (6): [' my talk page', ' talk page and', ' to edit the', " you're talking", ', and i', ".\nit's"]
4-grams (1): [' asked you to stop']


In [8]:
print(f"""
<DOC>
{known_text}
</DOC>
<NGRAM>
""")


<DOC>
We aren't on the best of terms, so I'll resist a pompous lecture on why that's 'really' not kosher.
Anyway, I see you're unblocked now, so at least one mind-numbingly unproductive ANI thread has been closed.
Goldrawer7, I'm sorry, but I've blocked this account from editing.
There are too many problems with your editing, and I believe you don't currently have the skills to edit the encyclopedia.
I've also blocked talk page and email access, as I don't believe an unblock request is something you should do on-wiki in this case.
Now, as an active, uninvolved admin, please block all ArbCom candidates except me, so they are ineligible.
Well, OK, and maybe 28bytes, he doesn't scare me.
Just got back from dinner at our friends' house, and I'm about to have another slice of mince meat pie that we brought home.
I think I like that more than the turkey.
Thanks, Ross, I appreciate the concern about my talk page.
But just so this doesn't turn into a bigger deal than it needs to be, it's OK t

## Phrase 1 -  " asked you to stop"

In [11]:
p_1 = " asked you to stop"

para_1 = [" asked you to cease", " asked you to halt", " asked you to desist", " requested you to stop", " requested you to cease", " requested you to halt", " requested you to desist", " requested that you stop", " requested that you cease", " requested that you halt", " requested that you desist", " made a request to you to stop", " made a request to you to cease", " made a request to you to halt", " made a request to you to desist", " made you a request to stop", " made you a request to cease", " made you a request to halt", " made you a request to desist", " made a request that you stop", " made a request that you cease", " made a request that you halt", " made a request that you desist", " sent you a request to stop", " sent you a request to cease", " sent you a request to halt", " sent you a request to desist", " sent a request to you to stop", " sent a request to you to cease", " sent a request to you to halt", " sent a request to you to desist", " sent a request that you stop", " sent a request that you cease", " sent a request that you halt", " sent a request that you desist", " asked ya to stop", " asked u to stop", " askd you to stop", " aksed you to stop"]

known_base_p1 = keep_before_phrase(known_text, p_1, True)
unknown_base_p1 = keep_before_phrase(unknown_text, p_1, True)

In [12]:
known_p1_scores = score_phrases(known_base_p1, p_1, para_1, tokenizer, model)
unknown_p1_scores = score_phrases(unknown_base_p1, p_1, para_1, tokenizer, model)

→ Scoring base_text alone…
   base_total = -1099.7274

→ [1/40] Processing reference…
→ [2/40] Processing paraphrase…
→ [3/40] Processing paraphrase…
→ [4/40] Processing paraphrase…
→ [5/40] Processing paraphrase…
→ [6/40] Processing paraphrase…
→ [7/40] Processing paraphrase…
→ [8/40] Processing paraphrase…
→ [9/40] Processing paraphrase…
→ [10/40] Processing paraphrase…
→ [11/40] Processing paraphrase…
→ [12/40] Processing paraphrase…
→ [13/40] Processing paraphrase…
→ [14/40] Processing paraphrase…
→ [15/40] Processing paraphrase…
→ [16/40] Processing paraphrase…
→ [17/40] Processing paraphrase…
→ [18/40] Processing paraphrase…
→ [19/40] Processing paraphrase…
→ [20/40] Processing paraphrase…
→ [21/40] Processing paraphrase…
→ [22/40] Processing paraphrase…
→ [23/40] Processing paraphrase…
→ [24/40] Processing paraphrase…
→ [25/40] Processing paraphrase…
→ [26/40] Processing paraphrase…
→ [27/40] Processing paraphrase…
→ [28/40] Processing paraphrase…
→ [29/40] Processing paraphrase

In [13]:
known_p1_pmf = add_pmf_column(known_p1_scores, 'phrase_log_probs')
unknown_p1_pmf = add_pmf_column(unknown_p1_scores, 'phrase_log_probs')

In [14]:
p1_pmf = score_phrases_no_context(p_1, para_1, tokenizer, model)
p1_pmf = add_pmf_column(p1_pmf, 'log_probs')

→ [1/40] Processing reference…
→ [2/40] Processing paraphrase…
→ [3/40] Processing paraphrase…
→ [4/40] Processing paraphrase…
→ [5/40] Processing paraphrase…
→ [6/40] Processing paraphrase…
→ [7/40] Processing paraphrase…
→ [8/40] Processing paraphrase…
→ [9/40] Processing paraphrase…
→ [10/40] Processing paraphrase…
→ [11/40] Processing paraphrase…
→ [12/40] Processing paraphrase…
→ [13/40] Processing paraphrase…
→ [14/40] Processing paraphrase…
→ [15/40] Processing paraphrase…
→ [16/40] Processing paraphrase…
→ [17/40] Processing paraphrase…
→ [18/40] Processing paraphrase…
→ [19/40] Processing paraphrase…
→ [20/40] Processing paraphrase…
→ [21/40] Processing paraphrase…
→ [22/40] Processing paraphrase…
→ [23/40] Processing paraphrase…
→ [24/40] Processing paraphrase…
→ [25/40] Processing paraphrase…
→ [26/40] Processing paraphrase…
→ [27/40] Processing paraphrase…
→ [28/40] Processing paraphrase…
→ [29/40] Processing paraphrase…
→ [30/40] Processing paraphrase…
→ [31/40] Processing

In [18]:
# unknown_p1_pmf.to_clipboard()

## Phrase 2 - " my talk page"

In [19]:
p_2 = " my talk page"

para_2 = [" my discussion page", " my talk-page", " my talkpage", " my discussion-page", " my discussionpage", " the talk page of mine", " the discussion page of mine", " my tlak page", " my talk pag", " my talk paeg", " my talkpgae", " my discusson page", " my discusion page", " my discussin page", " my discussion pg", " my talk pg"]

known_base_p2 = keep_before_phrase(known_text, p_2, True)
unknown_base_p2 = keep_before_phrase(unknown_text, p_2, True)

In [20]:
known_p2_scores = score_phrases(known_base_p2, p_2, para_2, tokenizer, model)
unknown_p2_scores = score_phrases(unknown_base_p2, p_2, para_2, tokenizer, model)

→ Scoring base_text alone…
   base_total = -884.9681

→ [1/17] Processing reference…
→ [2/17] Processing paraphrase…
→ [3/17] Processing paraphrase…
→ [4/17] Processing paraphrase…
→ [5/17] Processing paraphrase…
→ [6/17] Processing paraphrase…
→ [7/17] Processing paraphrase…
→ [8/17] Processing paraphrase…
→ [9/17] Processing paraphrase…
→ [10/17] Processing paraphrase…
→ [11/17] Processing paraphrase…
→ [12/17] Processing paraphrase…
→ [13/17] Processing paraphrase…
→ [14/17] Processing paraphrase…
→ [15/17] Processing paraphrase…
→ [16/17] Processing paraphrase…
→ [17/17] Processing paraphrase…
→ Scoring base_text alone…
   base_total = -249.5002

→ [1/17] Processing reference…
→ [2/17] Processing paraphrase…
→ [3/17] Processing paraphrase…
→ [4/17] Processing paraphrase…
→ [5/17] Processing paraphrase…
→ [6/17] Processing paraphrase…
→ [7/17] Processing paraphrase…
→ [8/17] Processing paraphrase…
→ [9/17] Processing paraphrase…
→ [10/17] Processing paraphrase…
→ [11/17] Processing 

In [21]:
known_p2_pmf = add_pmf_column(known_p2_scores, 'phrase_log_probs')
unknown_p2_pmf = add_pmf_column(unknown_p2_scores, 'phrase_log_probs')

In [22]:
p2_pmf = score_phrases_no_context(p_2, para_2, tokenizer, model)
p2_pmf = add_pmf_column(p2_pmf, 'log_probs')

→ [1/17] Processing reference…
→ [2/17] Processing paraphrase…
→ [3/17] Processing paraphrase…
→ [4/17] Processing paraphrase…
→ [5/17] Processing paraphrase…
→ [6/17] Processing paraphrase…
→ [7/17] Processing paraphrase…
→ [8/17] Processing paraphrase…
→ [9/17] Processing paraphrase…
→ [10/17] Processing paraphrase…
→ [11/17] Processing paraphrase…
→ [12/17] Processing paraphrase…
→ [13/17] Processing paraphrase…
→ [14/17] Processing paraphrase…
→ [15/17] Processing paraphrase…
→ [16/17] Processing paraphrase…
→ [17/17] Processing paraphrase…


In [26]:
# unknown_p2_pmf.to_clipboard()

## Phrase 3 - " to edit the"

In [27]:
p_3 = " to edit the"

para_3 = [" to modify the", " to amend the", " to revise the", " to alter the", " to change the", " to update the", " to emend the", " to redraft the", " to make an edit to the", " to make edits to the", " to make a revision to the", " to make revisions to the", " to make a change to the", " to make changes to the", " to make an amendment to the", " to make amendments to the", " to make a modification to the", " to make modifications to the", " to make an update to the", " to make updates to the", " to do an edit to the", " to do edits to the", " to perform an edit on the", " to perform edits on the", " to edit teh", " to edit hte", " to edti the", " to edit th", " to edit tje"]

known_base_p3 = keep_before_phrase(known_text, p_3, True)
unknown_base_p3 = keep_before_phrase(unknown_text, p_3, True)

In [28]:
known_p3_scores = score_phrases(known_base_p3, p_3, para_3, tokenizer, model)
unknown_p3_scores = score_phrases(unknown_base_p3, p_3, para_3, tokenizer, model)

→ Scoring base_text alone…
   base_total = -373.8153

→ [1/30] Processing reference…
→ [2/30] Processing paraphrase…
→ [3/30] Processing paraphrase…
→ [4/30] Processing paraphrase…
→ [5/30] Processing paraphrase…
→ [6/30] Processing paraphrase…
→ [7/30] Processing paraphrase…
→ [8/30] Processing paraphrase…
→ [9/30] Processing paraphrase…
→ [10/30] Processing paraphrase…
→ [11/30] Processing paraphrase…
→ [12/30] Processing paraphrase…
→ [13/30] Processing paraphrase…
→ [14/30] Processing paraphrase…
→ [15/30] Processing paraphrase…
→ [16/30] Processing paraphrase…
→ [17/30] Processing paraphrase…
→ [18/30] Processing paraphrase…
→ [19/30] Processing paraphrase…
→ [20/30] Processing paraphrase…
→ [21/30] Processing paraphrase…
→ [22/30] Processing paraphrase…
→ [23/30] Processing paraphrase…
→ [24/30] Processing paraphrase…
→ [25/30] Processing paraphrase…
→ [26/30] Processing paraphrase…
→ [27/30] Processing paraphrase…
→ [28/30] Processing paraphrase…
→ [29/30] Processing paraphrase…

In [29]:
known_p3_pmf = add_pmf_column(known_p3_scores, 'phrase_log_probs')
unknown_p3_pmf = add_pmf_column(unknown_p3_scores, 'phrase_log_probs')

In [30]:
p3_pmf = score_phrases_no_context(p_3, para_3, tokenizer, model)
p3_pmf = add_pmf_column(p3_pmf, 'log_probs')

→ [1/30] Processing reference…
→ [2/30] Processing paraphrase…
→ [3/30] Processing paraphrase…
→ [4/30] Processing paraphrase…
→ [5/30] Processing paraphrase…
→ [6/30] Processing paraphrase…
→ [7/30] Processing paraphrase…
→ [8/30] Processing paraphrase…
→ [9/30] Processing paraphrase…
→ [10/30] Processing paraphrase…
→ [11/30] Processing paraphrase…
→ [12/30] Processing paraphrase…
→ [13/30] Processing paraphrase…
→ [14/30] Processing paraphrase…
→ [15/30] Processing paraphrase…
→ [16/30] Processing paraphrase…
→ [17/30] Processing paraphrase…
→ [18/30] Processing paraphrase…
→ [19/30] Processing paraphrase…
→ [20/30] Processing paraphrase…
→ [21/30] Processing paraphrase…
→ [22/30] Processing paraphrase…
→ [23/30] Processing paraphrase…
→ [24/30] Processing paraphrase…
→ [25/30] Processing paraphrase…
→ [26/30] Processing paraphrase…
→ [27/30] Processing paraphrase…
→ [28/30] Processing paraphrase…
→ [29/30] Processing paraphrase…
→ [30/30] Processing paraphrase…


In [33]:
unknown_p3_pmf.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
